<a href="https://colab.research.google.com/github/auramolina/Analitica-en-recursos-humanos/blob/main/04%20Despliegue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Conectar el drive
from google.colab import drive
drive.mount('/content/drive')

import sys
import os

path='/content/drive/MyDrive/AA/Analitica-en-recursos-humanos'
os.chdir(path) ### setting a Working directory on drive path
sys.path.append(path+'/')  ###setting path to read user defined functions

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import funciones_jc as funciones  ###archivo de funciones propias
import pandas as pd ### para manejo de datos
# import sqlite3 as sql
import joblib
import openpyxl ## para exportar a excel
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [12]:
# Cargar objeto guardado
d2016 = joblib.load("/content/drive/MyDrive/AA/Analitica-en-recursos-humanos/salidas/ddummies2016.pkl")

In [17]:
###### el despliegue consiste en dejar todo el código listo para una ejecucion automática en el periodo definido:
###### en este caso se ejecutara el proceso de entrenamiento y prediccion anualmente.
if __name__=="__main__":

  d2016

  #### Otras transformaciones en python (imputación, dummies y selección de variables)
  df_t = funciones.preparar_datos(d2016)

  ## Cargar modelo y predecir
  final = joblib.load("/content/drive/MyDrive/AA/Analitica-en-recursos-humanos/salidas/gb_final.pkl")
  predicciones = final.predict(df_t)
  pd_pred = pd.DataFrame(predicciones, columns=['Retiros'])

  ### Crear base con predicciones ###
  pred = pd.concat([d2016['EmployeeID'], df_t, pd_pred], axis=1)

  ####LLevar a BD para despliegue
  ret = pred[['EmployeeID', 'Retiros']]
  #ret.to_csv("/content/drive/MyDrive/AA/Analitica-en-recursos-humanos/salidas/pred.csv", index=False)
  ret2016 = ret.sort_values(by='EmployeeID', ascending=True)
  ret2016.to_excel("/content/drive/MyDrive/AA/Analitica-en-recursos-humanos/salidas/pred2016.xlsx", index=False)
  ret_yes = ret.query('Retiros == 1')
  ret_yes.to_excel("/content/drive/MyDrive/AA/Analitica-en-recursos-humanos/salidas/retiros.xlsx", index=False)